<a href="https://colab.research.google.com/github/JimKing100/NFL-Live/blob/master/data-science/metrics/MetricsNN_PvA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Imports
import pandas as pd
import numpy as np

In [0]:
# Load the raw data
w1_results_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/NFL-Live/master/data-science/data/rnn-combined/predictions-week1.csv')

In [3]:
#### The week 1 predictions

# week1-cur = 2018 total points
# week1-pred = predicted points for the season
# week1-act = actual points for the season

# weekn-cur = week (n-1) actual points
# weekn-pred = predicted points for the rest of the season (n-17)
# weekn-act = actual points for the rest of the season (n-17)

w1_results_df.head()

,player,first,last,name,position,week1-cur,week1-pred,week1-act,week1-diff,week1-pct,rank-cur,rank-pred,rank-act
0,SB-0212,Saquon,Barkley,Saquon Barkley,RB,304.80,323,196.10,127.750000,0.394473,9.0,1.0,36.0
1,RW-3850,Russell,Wilson,Russell Wilson,QB,301.42,310,333.64,-22.906921,-0.073719,10.0,2.0,4.0
2,DP-2037,Dak,Prescott,Dak Prescott,QB,290.90,299,344.74,-45.667500,-0.152697,12.0,3.0,3.0
3,AR-1300,Aaron,Rodgers,Aaron Rodgers,QB,317.58,297,282.38,15.275935,0.051321,5.0,4.0,11.0
4,CN-0500,Cam,Newton,Cam Newton,QB,283.60,291,16.68,275.042487,0.942822,14.0,5.0,424.0


In [4]:
# Calculate the metrics for NN vs Baseline Average
column_names = ['week', 'avg-total', 'avg-pct', 'nn-total', 'nn-pct', 'nb-correct']
metric_df = pd.DataFrame(columns = column_names)

for i in range(1, 18):

    filename = 'https://raw.githubusercontent.com/JimKing100/NFL-Live/master/data-science/data/rnn-combined/predictions-week' + str(i) + '.csv'
    
    # Column names
    week_cur = 'week' + str(i) + '-cur'
    week_pred = 'week' + str(i) + '-pred'
    week_act = 'week' + str(i) + '-act'

    # Weekly predictions
    results_df = pd.read_csv(filename)

    # Create the current points list using 2018 points in week 1 and average points going forward
    if i == 1:
        week_current = results_df['week1-cur'].tolist()
    else:
        # for each player (element) calculate the average points (element/(i-1)) and multiply by remaining games (17-(i-1))
        # the 17th week is 0 and represents the bye week (17 weeks and 16 games)
        week_list = results_df[week_cur].tolist()
        week_current = [((element / (i -1)) * (17 - (i -1))) for element in week_list]

    # Create the prediction and actual lists
    week_pred = results_df[week_pred].tolist()
    week_act = results_df[week_act].tolist()

    # Calulate prediction vs. actual for average
    
    cur_total_correct = 0
    num_players = len(week_current)

    for j in range(0, num_players):
        cur_count = 0
        act_count = 0
        for k in range(0, num_players):
          if j != k:
              if week_current[j] < week_current[k]:
                  cur_count = cur_count + 1
              if week_act[j] < week_act[k]:
                  act_count = act_count + 1
        cur_total_correct = cur_total_correct + (num_players - abs(act_count - cur_count))
        # print(cur_count, act_count, num_players, '# correct predictions = ', (num_players - abs(act_count - cur_count)), cur_total_correct)
    print('Total correct and % correct - Baseline Average', cur_total_correct, cur_total_correct/(653 * 652))

    # Calulate prediction vs. actual for ARIMA
    
    pred_total_correct = 0
    num_players = len(week_pred)

    for j in range(0, num_players):
        pred_count = 0
        act_count = 0
        for k in range(0, num_players):
          if j != k:
              if week_pred[j] < week_pred[k]:
                  pred_count = pred_count + 1
              if week_act[j] < week_act[k]:
                  act_count = act_count + 1
        pred_total_correct = pred_total_correct + (num_players - abs(act_count - pred_count))
        # print(pred_count, act_count, num_players, '# correct predictions = ', (num_players - abs(act_count - pred_count)), pred_total_correct)
    print('Total correct and % correct - NN', pred_total_correct, pred_total_correct/(653 * 652))
    print('Total additional correct using NN = ', pred_total_correct - cur_total_correct)

    metric_df = metric_df.append({'week': i, 'avg-total': cur_total_correct, 'avg-pct': cur_total_correct/(653 * 652),
                                  'nn-total': pred_total_correct, 'nn-pct': pred_total_correct/(653 * 652),
                                  'nb-correct': pred_total_correct - cur_total_correct}, ignore_index=True)

file_name = '/content/nb_metrics.csv'
metric_df.to_csv(file_name, index=False)

Total correct and % correct - Baseline Average 350701 0.8237135824274937
Total correct and % correct - NN 361235 0.8484554533582616
Total additional correct using NN =  10534
Total correct and % correct - Baseline Average 341356 0.8017643908717669
Total correct and % correct - NN 363323 0.8533596707973581
Total additional correct using NN =  21967
Total correct and % correct - Baseline Average 350501 0.8232438297992277
Total correct and % correct - NN 364020 0.854996758706865
Total additional correct using NN =  13519
Total correct and % correct - Baseline Average 355994 0.8361455857345522
Total correct and % correct - NN 364666 0.856514059696164
Total additional correct using NN =  8672
Total correct and % correct - Baseline Average 357082 0.8387010400323189
Total correct and % correct - NN 364327 0.8557178289912533
Total additional correct using NN =  7245
Total correct and % correct - Baseline Average 357546 0.839790866129896
Total correct and % correct - NN 363324 0.853362019560499